# CORE: Feature Engineering 
Name: Jude Maico Jr.

Your task is to engineer some new features to try to improve a model's ability to predict the total number of bike share rentals during a given hour of the day.

1. Import the data the drop the 'casual' and 'registered' columns. These are redundant with your target, 'count'.
2. Transform the 'datetime' column into a datetime type and use it to create 3 new columns in the data frame containing the:
    1. Name of the Month
    2. Name of the Day of the Week
    3. Hour of the Day
        1. Make sure all 3 new columns are 'object' datatype so they can be one-hot encoded later.
        2. Drop the 'datetime' and 'season' columns. These are now redundant.
3. The temperatures in the 'temp' and 'atemp' columns are in Celsius. Use `.apply()` and a Lambda function to convert them to Fahrenheit.
5. Create a new column, 'temp_variance' which shows how much warmer or colder the current temperature ('temp') is than the average temperate for that day of the year ('atemp').  If the current temperature is warmer than average ('atemp'), the value in 'temp_variance' should be positive. 
    1. Drop the 'atemp' column.
    
Optional:
- Use a predictive model of your choice and try to predict the 'count' of hourly bike-share users with both the original features and the engineered feature set you created.

- Remember to drop the 'casual' and 'registered' columns from both versions before modeling.

- Did these feature engineering choices improve your ability to predict the 'count'?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv(r'D:\Data Science\DATA\bikeshare_train - bikeshare_train.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df = df.drop(columns = ['casual', 'registered'])
df.sample(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
666,2011-02-11 0:00:00,1,0,1,1,4.10,6.820,49,7.0015,14
3935,2011-09-15 0:00:00,3,0,1,1,26.24,30.305,69,8.9981,37
10314,2012-11-15 4:00:00,4,0,1,2,11.48,14.395,65,8.9981,5


In [4]:
# for optional assignment
df2 = df.copy()
df2.sample(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
4015,2011-09-18 8:00:00,3,0,0,1,18.86,22.725,72,16.9979,69
8735,2012-08-06 8:00:00,3,0,1,2,28.70,33.335,79,0.0000,605
4471,2011-10-18 8:00:00,4,0,1,1,20.50,24.240,77,6.0032,499


In [5]:
df['datetime'] = pd.to_datetime(df['datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(6)
memory usage: 850.6 KB


In [6]:
#name of the month
df['month'] = df['datetime'].dt.month_name()

#day of the week
df['week_day'] = df['datetime'].dt.day_name()

#time of the day
df['day_time'] = (df['datetime'].dt.hour % 24 + 4) // 4
df['day_time'].replace({1: 'Late Night',
                        2: 'Early Morning',
                        3: 'Morning',
                        4: 'Noon',
                        5: 'Evening',
                        6: 'Night'}, inplace=True)

df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,week_day,day_time
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,16,January,Saturday,Late Night
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,40,January,Saturday,Late Night
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,32,January,Saturday,Late Night
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,13,January,Saturday,Late Night
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,1,January,Saturday,Early Morning


In [7]:
df.drop(columns = ['datetime', 'season'], inplace = True)
df.columns

Index(['holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity',
       'windspeed', 'count', 'month', 'week_day', 'day_time'],
      dtype='object')

In [8]:
df[['temp','atemp']] = df[['temp', 'atemp']].apply(lambda x: (1.8 * x) +32)
df.rename(columns = {'temp' : 'fahrenheit_temp'}, inplace = True)
df.head()

,holiday,workingday,weather,fahrenheit_temp,atemp,humidity,windspeed,count,month,week_day,day_time
0,0,0,1,49.712,57.911,81,0.0,16,January,Saturday,Late Night
1,0,0,1,48.236,56.543,80,0.0,40,January,Saturday,Late Night
2,0,0,1,48.236,56.543,80,0.0,32,January,Saturday,Late Night
3,0,0,1,49.712,57.911,75,0.0,13,January,Saturday,Late Night
4,0,0,1,49.712,57.911,75,0.0,1,January,Saturday,Early Morning


In [9]:
median_temp = df['atemp'].median()

def thermal(temp):
    if temp > median_temp:
        return 'Warm'
    else:
        return 'Cold'
    
df['atemp'] = df['atemp'].apply(thermal)
df.rename(columns = {'atemp' : 'temp_variance'}, inplace = True)
df.sample(5)

,holiday,workingday,weather,fahrenheit_temp,temp_variance,humidity,windspeed,count,month,week_day,day_time
2084,0,1,3,70.376,Warm,88,11.0014,398,May,Friday,Evening
2484,0,0,1,86.612,Warm,58,11.0014,264,June,Saturday,Morning
4084,0,0,1,55.616,Cold,87,19.0012,7,October,Sunday,Early Morning
8898,0,1,1,79.232,Warm,69,0.0000,6,August,Monday,Late Night
5994,0,0,1,49.712,Cold,70,8.9981,42,February,Sunday,Night


# Optional Assignment:

## Preparing Dataset of ML

In [10]:
data_types = df.dtypes
object_data_types = data_types[(data_types == 'object')]

for column in object_data_types.index:
  print(column)
  print('\n')
  print(f'Unique Values: \n{df[column].unique()}')
  print('\n')

temp_variance


Unique Values: 
['Cold' 'Warm']


month


Unique Values: 
['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']


week_day


Unique Values: 
['Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday']


day_time


Unique Values: 
['Late Night' 'Early Morning' 'Morning' 'Noon' 'Evening' 'Night']




In [11]:
data_types = df2.dtypes
object_data_types = data_types[(data_types == 'object')]

for column in object_data_types.index:
  print(column)
  print('\n')
  print(f'Unique Values: \n{df2[column].unique()}')
  print('\n')

datetime


Unique Values: 
['2011-01-01 0:00:00' '2011-01-01 1:00:00' '2011-01-01 2:00:00' ...
 '2012-12-19 21:00:00' '2012-12-19 22:00:00' '2012-12-19 23:00:00']




In [12]:
df.describe()

,holiday,workingday,weather,fahrenheit_temp,humidity,windspeed,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,0.028569,0.680875,1.418427,68.415548,61.886460,12.799395,191.574132
std,0.166599,0.466159,0.633839,14.024862,19.245033,8.164537,181.144454
min,0.000000,0.000000,1.000000,33.476000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,1.000000,57.092000,47.000000,7.001500,42.000000
50%,0.000000,1.000000,1.000000,68.900000,62.000000,12.998000,145.000000
75%,0.000000,1.000000,2.000000,79.232000,77.000000,16.997900,284.000000
max,1.000000,1.000000,4.000000,105.800000,100.000000,56.996900,977.000000


In [13]:
df2.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,977.000000


In [14]:
df.duplicated().sum()

18

In [15]:
#drop duplicates in first model
df.drop_duplicates(inplace = True)

In [16]:
#drop duplicates in second model
df2.drop_duplicates(inplace = True)

In [17]:
#final check on first model
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10868 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   holiday          10868 non-null  int64  
 1   workingday       10868 non-null  int64  
 2   weather          10868 non-null  int64  
 3   fahrenheit_temp  10868 non-null  float64
 4   temp_variance    10868 non-null  object 
 5   humidity         10868 non-null  int64  
 6   windspeed        10868 non-null  float64
 7   count            10868 non-null  int64  
 8   month            10868 non-null  object 
 9   week_day         10868 non-null  object 
 10  day_time         10868 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 1018.9+ KB


,holiday,workingday,weather,fahrenheit_temp,temp_variance,humidity,windspeed,count,month,week_day,day_time
0,0,0,1,49.712,Cold,81,0.0,16,January,Saturday,Late Night
1,0,0,1,48.236,Cold,80,0.0,40,January,Saturday,Late Night
2,0,0,1,48.236,Cold,80,0.0,32,January,Saturday,Late Night


In [18]:
#final check on second model
df2.info()
df2.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10886 entries, 0 to 10885
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   count       10886 non-null  int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 935.5+ KB


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32


Model 1: </br>
**Categorical Features**: holiday, workingday, weather </br>
**Nominal Features**: temp_variance, month, week_day, day_time </br>
**Numerical Features**: fahrenheit_temp, humidity, windspeed

Model 2: </br> 
**Categorical Features**: season, holiday, workingday, weather </br>
**Nominal Features**: datetime</br>
**Numerical Features**: temp, atemp, humidity, windspeed</br>

Target is a continuous number - **Regression Model** will be used.

## Model 1 (Featured Engineered)

In [19]:
target = 'count'
y = df[target]
X = df.drop(columns = [target])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [20]:
# Categorical features: already in numerical 
cat_cols = ['holiday', 'workingday', 'weather']
cat_scale = StandardScaler()
cat_tuples = (cat_scale, cat_cols)

In [21]:
# Nominal features
nom_cols = ['temp_variance', 'month', 'week_day', 'day_time']
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
nom_tuples = (ohe, nom_cols)

In [22]:
# Numerical features
num_cols = ['fahrenheit_temp', 'humidity', 'windspeed']
num_scaler = StandardScaler()
num_tuples = (num_scaler, num_cols)

In [23]:
preprocessor = make_column_transformer(cat_tuples, nom_tuples, num_tuples,
                                       remainder = 'drop')

In [24]:
def performance(model, X_train, X_test, y_train, y_test, 
                preprocessor_number, name):
    
    #pipeline
    model_pipe = make_pipeline(preprocessor_number, model)
    model_pipe.fit(X_train, y_train)
    
    #prediction
    model_test = model_pipe.predict(X_test)
    model_train = model_pipe.predict(X_train)
    
    #train score
    mae_train = mean_absolute_error(y_train, model_train)
    mse_train = mean_squared_error(y_train, model_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y_train, model_train)     
    
    #prediction score
    mae_test = mean_absolute_error(y_test, model_test)
    mse_test = mean_squared_error(y_test, model_test)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_test, model_test)   
    
    print(f'{name} Trained Scores:')
    print(f'R^2: {r2_train:.3f} \nMAE: {mae_train:.3f}')
    print(f'MSE: {mse_train:.3f} \nRMSE: {rmse_train:.3f} \n')
          
    print(f'{name} Test Scores:')
    print(f'R^2: {r2_test:.3f} \nMAE: {mae_test:.3f}')
    print(f'MSE: {mse_test:.3f} \nRMSE: {rmse_test:.3f}')

In [25]:
linear = LinearRegression()
performance(linear, X_train, X_test, y_train, y_test,
            preprocessor, name = 'Engineered Linear Regression', )

Engineered Linear Regression Trained Scores:
R^2: 0.527 
MAE: 91.394
MSE: 15497.150 
RMSE: 124.488 

Engineered Linear Regression Test Scores:
R^2: 0.541 
MAE: 91.030
MSE: 15149.867 
RMSE: 123.085


## Model 2

In [26]:
y2 = df2[target]
X2 = df2.drop(columns = [target])
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, 
                                                        random_state = 42)

In [27]:
# Categorical features for second model
cat_cols2 = ['season', 'holiday', 'workingday', 'weather']
cat_scale2 = StandardScaler()
cat_tuples2 = (cat_scale2, cat_cols2)

In [28]:
# Nominal features for second model
nom_cols2 = ['datetime']
ohe2 = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
nom_tuples2 = (ohe2, nom_cols2)

In [29]:
# Numerical features for sencond model
num_cols2 = ['temp', 'atemp', 'humidity', 'windspeed']
num_scaler2 = StandardScaler()
num_tuples2 = (num_scaler2, num_cols2)

In [30]:
preprocessor2 = make_column_transformer(cat_tuples2, nom_tuples2, num_tuples2,
                                        remainder = 'drop')

In [31]:
linear2 = LinearRegression()
performance(linear2, X2_train, X2_test, y2_train, y2_test,
            preprocessor2, name = 'Defaul Model Linear Regression')

Defaul Model Linear Regression Trained Scores:
R^2: 1.000 
MAE: 0.000
MSE: 0.000 
RMSE: 0.000 

Defaul Model Linear Regression Test Scores:
R^2: 0.207 
MAE: 122.742
MSE: 26019.771 
RMSE: 161.306


Observations: </br>
- We have overfitting in the default dataset: we see that the model did perfect on training set while worst on the testing set.
- The testing set on Engineered model was able to predict 54% of our data set. A much higher prediction than the default data set. 
- Lowest MAE, MSE and RMSE are observed in testing for Engineered model. 
